# loading pretrained tokenizer and text classification model

In [67]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained(
    'bhadresh-savani/bert-base-uncased-emotion')


# defining dummy input

In [77]:
dummy_input_texts = [
    'My name is Clara and I live in Berkeley, California. I work at this cool company called Hugging Face.',
    'Hello, my dog is cute.',
    'This is bad.',
    'I like you. I love you'
]

dummy_inputs = tokenizer(
    dummy_input_texts, return_tensors="pt", padding=True, truncation=True)
print('\n'.join([f'text{idx} -> {tokenizer.decode(text)}' for idx,
      text in enumerate(dummy_inputs['input_ids'])]))

# batch_size = len(dummy_input_texts)
# dummy_inputs['labels'] = torch.full((1, batch_size), model.num_labels-1)


text0 -> [CLS] my name is clara and i live in berkeley, california. i work at this cool company called hugging face. [SEP]
text1 -> [CLS] hello, my dog is cute. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
text2 -> [CLS] this is bad. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
text3 -> [CLS] i like you. i love you [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


# feeding dummy input to PyTorch model

In [78]:
outputs = model(**dummy_inputs)

print('\n'.join([f'text{idx} -> {model.config.id2label[int(logit.argmax())]}:\
      {float(logit.softmax(dim=-1).max())*100:.2f}%'
      for idx, logit in enumerate(outputs.logits)]))


text0 -> joy:      99.87%
text1 -> joy:      99.79%
text2 -> sadness:      72.95%
text3 -> joy:      74.21%


# exporting PyTorch model to ONNX format

In [ ]:
from transformers.models.bert import BertOnnxConfig

config = model.config
onnx_config = BertOnnxConfig(config)

input_names = list(onnx_config.inputs.keys())
input_values = tuple([dummy_inputs[key] for key in input_names])
output_names = list(outputs.keys())
output_path = 'onnx/bert-base-uncased/bert-base-uncased-emotion.onnx'

torch.onnx.export(model, input_values, output_path, input_names=input_names, output_names=output_names, opset_version=10)


# feeding dummy input to ONNX model

In [ ]:
import onnxruntime as ort
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
ort_session = ort.InferenceSession("onnx/bert-base-uncased/bert-base-uncased-emotion.onnx")

inputs = tokenizer(dummy_input_texts, return_tensors="np",
                   padding=True, truncation=True)
outputs = ort_session.run(output_names, dict(inputs))


# printing ONNX model's output

In [74]:
print('\n'.join([f'text{idx} -> {model.config.id2label[int(output.argmax())]}:\
      {float(torch.Tensor(output).softmax(dim=-1).max())*100:.2f}%'
      for idx, output in enumerate(torch.Tensor(outputs)[0])]))


text0 -> joy:      99.87%
text1 -> joy:      99.79%
text2 -> sadness:      72.95%
text3 -> joy:      74.21%
